In [1]:
import re
import pandas as pd

In [27]:
string = '''select 
mhr.operator_name,
date_format(mhr.start_time,'%d') day,
pt.process_name,
prn.process_routing_code,
mhr.project_code,
sum(mhr.amount) sum,
prn.product_desc*1.0 jjdj,
if((sum(mhr.amount)*prn.product_desc*1.0) is null,0,sum(mhr.amount)*prn.product_desc*1.0) jjgz
FROM manufacture.manual_hold_record mhr
left join manufacture.produce_task pt
on pt.org_id=mhr.org_id and pt.id=mhr.task_id and pt.operator_ids->>'$[0]'=mhr.report_id
left join def.process_routing_node prn
on prn.org_id=mhr.org_id and prn.process_code=pt.process_code
join manufacture.project p
on p.org_id=mhr.org_id and p.project_code=mhr.project_code and prn.process_routing_code=p.process_routing_code 
left join 
(select org_id,
task_id,
date_format(json_unquote(json_extract(json_extract(content,'$[0].controls'),'$[0].value')),'%Y-%m-%d %H:%i:%S') huanmo_stime,
date_format(json_unquote(json_extract(json_extract(content,'$[0].controls'),'$[1].value')),'%Y-%m-%d %H:%i:%S') huanmo_etime,
json_unquote(json_extract(json_unquote(json_extract(json_extract(content,'$[0].controls'),'$[2].value')),'$.userName')) people
from manufacture.produce_report pr
where org_id=610388
) pr
on mhr.org_id=pr.org_id and mhr.task_id=pr.task_id and =pr.people
where pt.org_id=610388 and pt.operator_group_id=822 and date(mhr.start_time)=date(mhr.end_time)
 and DATE_FORMAT(date(mhr.start_time), '%Y-%m' ) = DATE_FORMAT(str_to_date('${time}','%Y-%m'), '%Y-%m' )
-- and DATE_FORMAT(date(mhr.end_time), '%Y-%m' ) = DATE_FORMAT(str_to_date('${time}','%Y-%m'), '%Y-%m' )
group by ,date_format(mhr.start_time,'%d'),prn.process_routing_code,prn.product_desc*1.0,mhr.project_code,pt.process_name 


'''

In [28]:
pat1 = re.compile(r'FROM +((?!\()\w+?)\.')
pat2 = re.compile(r'FROM\n\t*((?!\()\w+?)\.')
pat3 = re.compile(r'LEFT JOIN +((?!\()\w+?)\.')
pat3_AS = re.compile(r'LEFT JOIN +AS ((?!\()\w+?)\.')

pat1l = re.compile(r'from +((?!\()\w+?)\.')
pat2l = re.compile(r'from\n*\t*((?!\()\w+?)\.')
pat3l = re.compile(r'left join +((?!\()\w+?)\.')
pat3l_as = re.compile(r'left join +as ((?!\()\w+?)\.')
r = list(set(re.findall(pat1,string) + re.findall(pat2,string) + re.findall(pat1l,string) + re.findall(pat2l,string) + 
       re.findall(pat3,string) + re.findall(pat3_AS,string) + re.findall(pat3l,string)+ re.findall(pat3l_as,string)))
#提取库名

print ('库  :  表','\n')

for item in r:
    p = re.compile(item+r'\.([^ \n](?!\{)(?!\$)\w+)')
    temp = list(set(re.findall(p,string)))
    temp.sort()
    # 提取表名
    print (item,':',temp,'\n')
    temp.append('-------------------------')
    
    item = '库:'+item
    df2 = pd.DataFrame()
    df2 = pd.DataFrame(columns = [item], data = temp)
    df2.to_csv('/Users/data/desktop/table.csv', mode = 'a', index = False)
    # 输出「库：表」
    temp = temp[:-1]
    
    for table in temp:
        p_1 = re.compile(r'\.'+table+r' +((?!where)(?!WHERE)(?!as)(?!AS)\w+)')
        p_2 = re.compile(r'\.'+table+r' +AS (\w+)')
        p_2l = re.compile(r'\.'+table+r' +as (\w+)')
        alias = list(set(re.findall(p_1,string)+re.findall(p_2,string)+re.findall(p_2l,string)))
        if alias:
            print ('\t',table,':',alias,'\n')
        else:
            print ('\n\t',table,':未重命名','\n')
        #提取表的重命名
        
        ziduan = []
        
        if alias:
            for i in alias:
                p_3 = re.compile(r' '+i+'\.(\w+)')
                p_4 = re.compile(r', *'+i+'\.(\w+)')
                p_5 = re.compile(r'= *'+i+'\.(\w+)')
                p_6 = re.compile(r'\( *'+i+'\.(\w+)')
                p_7 = re.compile(r'\n *\t*'+i+'\.(\w+)')
                ziduan = list(set(ziduan+list(set(re.findall(p_3,string)+re.findall(p_4,string)
                                       +re.findall(p_5,string)+re.findall(p_6,string)+re.findall(p_7,string)))))
                ziduan = list(set(list(map(lambda x: x.lower(), ziduan ))))
            
            ziduan.sort() 
            #将字段按首字母进行排序
            
            print ('\t\t共',len(ziduan),'个字段:',ziduan)
            ziduan.append('-------------------------')
            
            table = '表:'+table
            df = pd.DataFrame()
            df = pd.DataFrame(columns = [table], data = ziduan)
            df.to_csv('/Users/data/desktop/ziduan.csv', mode = 'a', index = False)
            #提取字段并输出

库  :  表 

manufacture : ['manual_hold_record', 'produce_report', 'produce_task', 'project'] 

	 manual_hold_record : ['mhr'] 

		共 8 个字段: ['amount', 'end_time', 'operator_name', 'org_id', 'project_code', 'report_id', 'start_time', 'task_id']
	 produce_report : ['pr'] 

		共 3 个字段: ['org_id', 'people', 'task_id']
	 produce_task : ['pt'] 

		共 6 个字段: ['id', 'operator_group_id', 'operator_ids', 'org_id', 'process_code', 'process_name']
	 project : ['p'] 

		共 3 个字段: ['org_id', 'process_routing_code', 'project_code']
def : ['process_routing_node'] 

	 process_routing_node : ['prn'] 

		共 4 个字段: ['org_id', 'process_code', 'process_routing_code', 'product_desc']
